# Parametric model of plate with multiple holes

### Objective:
In this example we'll setup a parametric model in PyMAPDL for a rectangular plate with multiple holes. The model is setup such that one can change the dimensions of the plate, the number of holes and their radius, the material properties and the pressure applied.

The learning objectives of this demo are:
* Launch PyMAPDL on a local machine
* Setup and solve a parametric model using PyMAPDL
* Interactive plotting of CAD, mesh, and kresults in Pythonic interface.

#### Model parameters:
* Length, width and depth of the plate
* Number of holes
* Raidus of the holes
* Material properties (Young's modulus and Poisson's ratio)
* Applied pressure

# Step 1 - define all parameters

In [1]:
# All units in (m, Kg, s)
LENGTH = 5
WIDTH = 2.5
DEPTH = 0.1
RADIUS = 0.5
NUM = 3

E = 2e11
NU = 0.27

PRESSURE = 1000

# Step 2 - launch MAPDL and create geometry

In [2]:
from ansys.mapdl.core import launch_mapdl
mapdl = launch_mapdl()

mapdl.clear()
mapdl.prep7()
mapdl.block(0, LENGTH, 0, WIDTH, 0, DEPTH)
for i in range(1, NUM+1):
    mapdl.cyl4(i*LENGTH/(NUM+1), WIDTH/2, RADIUS, '', '', '', 2*DEPTH)

mapdl.vsbv(1, 'all')
mapdl.vplot('all')

e:\ansys\test\.venv\Lib\site-packages\pyvista\core\pointset.py:843: PyVistaDeprecationWarning: `PolyData` constructor parameter `n_faces` is deprecated and no longer used.
  warnings.warn(


Widget(value='<iframe src="http://localhost:11862/index.html?ui=P_0x2467b52a0f0_1&reconnect=auto" class="pyvis…

# Step 3 - define material properties, mesh attributes and generate mesh

In [3]:
mapdl.lesize("ALL", 0.15, layer1=1)

mapdl.mp('ex', 1, E)
mapdl.mp('nuxy', 1, NU)

mapdl.et(1, 'SOLID186')
mapdl.mshape(1, "3D")
mapdl.mshkey(0)
mapdl.vmesh('all')
mapdl.eplot()

Widget(value='<iframe src="http://localhost:11862/index.html?ui=P_0x2467d48c320_2&reconnect=auto" class="pyvis…

# Step 4 - apply loads and boundary conditions

In [4]:
mapdl.nsel('s', 'loc', 'x', 0)
mapdl.d('all', 'all', 0)

mapdl.nsel('s', 'loc', 'x', LENGTH)
mapdl.sf('all', 'pres', PRESSURE)

mapdl.allsel()
mapdl.finish()

***** ROUTINE COMPLETED *****  CP =         0.531

# Step 5 - solve the static problem

In [5]:
mapdl.slashsolu()
mapdl.solve()
mapdl.finish()

FINISH SOLUTION PROCESSING


 ***** ROUTINE COMPLETED *****  CP =         0.797

# Step 6 - plot the stress contours for the model

In [6]:
result = mapdl.result
result.plot_principal_nodal_stress(0, 'seqv', background='w', show_edges=True, text_color='k', add_text=True)

Widget(value='<iframe src="http://localhost:11862/index.html?ui=P_0x2467d4fb290_3&reconnect=auto" class="pyvis…

# Step 7 - exit MAPDL

In [7]:
mapdl.exit()